In [1]:
%%bash

gsutil cp -r gs://ot-snapshots/link-2/output/cooccurrences/part-00000-0883439b-5aa1-4062-a96f-fc871cb218b5-c000.snappy.parquet \
    /Users/dsuveges/project_data/epmc_evidence/

Copying gs://ot-snapshots/link-2/output/cooccurrences/part-00000-0883439b-5aa1-4062-a96f-fc871cb218b5-c000.snappy.parquet...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

- [1 files][231.4 MiB/231.4 MiB]    1.2 MiB/s                                   
Operation completed over 1 objects/231.4 MiB.                                    


In [15]:
import pyspark.sql
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import re

global spark

# SparkContext.setSystemProperty('spark.executor.memory', '20g')

spark = (pyspark.sql.SparkSession
    .builder
    .appName("phenodigm_parser")
    .config("spark.executor.memory", '10g')
     .config("spark.driver.bindAddress", "localhost")
    .config("spark.driver.memory", '10g')
    .getOrCreate()
)

#   

partitionKeys = ['pmid', 'targetFromSourceId', 'diseaseFromSourceMappedId']
print('Spark version: ', spark.version)
epmc_cooc_file = '/Users/dsuveges/project_data/epmc_evidence/cooccurrences/*'
# cooc_df = (
#     spark.read.parquet(epmc_cooc_file)
#     .filter((col('type') == "GP-DS") & (col('isMapped') == True) & (col('pmid') != ""))
#     .withColumnRenamed("keywordId1", "targetFromSourceId")
#     .withColumnRenamed("keywordId2", "diseaseFromSourceMappedId")
#     .withColumnRenamed("label1", "targetFromSource")
#     .withColumnRenamed("label2", "diseaseFromSource")
#     .dropDuplicates(partitionKeys)
#     .filter(col('targetFromSource') == '(')
#     .select('pmid', 'targetFromSource', 'targetFromSourceId', 'diseaseFromSourceMappedId', 'section', 'evidence_score')
#     .toPandas()
# )

partitionKeys = ['pmid', 'keywordId1', 'keywordId2']

cooc_df = (
    spark.read.json(epmc_cooc_file)
    .filter((col('type') == "GP-DS") & (col('isMapped') == True) & (col('pmid') != "") & (col('label1') == '('))
    .dropDuplicates(partitionKeys)
    .write.json('weird_names.json')
)


# 'pmid',
#  'pubDate',
#  'organisms',
#  'section',
#  'text',
#  'association',
#  'end1',
#  'end2',
#  'evidence_score',
#  'targetFromSource',
#  'targetFromSourceId',
#  'diseaseFromSource',
#  'diseaseFromSourceMappedId',
#  'relation',
#  'start1',
#  'start2',
#  'type',
#  'type1',
#  'type2',
#  'isMapped'
print(len(cooc_df))
cooc_df.head()

Spark version:  3.0.0


KeyboardInterrupt: 

In [3]:
from pyspark import *
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

import re

global spark

# SparkContext.setSystemProperty('spark.executor.memory', '20g')

sparkConf = (SparkConf()
         .set("spark.driver.memory", "15g")
         .set("spark.executor.memory", "15g")
         .set("spark.driver.maxResultSize", "0")
         .set("spark.debug.maxToStringFields", "2000")
         .set("spark.sql.execution.arrow.maxRecordsPerBatch", "500000")
         )
spark = (
    SparkSession.builder
        .config(conf=sparkConf)
         .config("spark.driver.bindAddress", "localhost")
        .master('local[*]')
        .getOrCreate()
)

cooccurrenceFile = '/Users/dsuveges/project_data/epmc_evidence/cooccurrences/*'
outputFile = '2021.03.18_epmc.json.gz'

partitionKeys = ['pmid', 'targetFromSourceId', 'diseaseFromSourceMappedId']
w = Window.partitionBy(*partitionKeys)
(
    # Reading file:
    spark.read.json(cooccurrenceFile)

    # Filtering for diases/target cooccurrences:
    .filter((col('type') == "GP-DS") & (col('isMapped') == True) & (col('pmid') != ""))

    # Renaming columns:
    .withColumnRenamed("keywordId1", "targetFromSourceId")
    .withColumnRenamed("keywordId2", "diseaseFromSourceMappedId")
    .withColumnRenamed("label1", "targetFromSource")
    .withColumnRenamed("label2", "diseaseFromSource")

        # collect sets of field values per window aggregation in w with keys partitionKeys
    .withColumn('textMiningSentences', collect_set(
            struct(
                col("text"),
                col('start1').alias('tStart'),
                col("end1").alias('tEnd'),
                col('start2').alias('dStart'),
                col("end2").alias('dEnd'),
                col('section')
            )).over(w)
        )
    .withColumn('resourceScore', sum(col('evidence_score')).over(w))
    .filter(col('resourceScore') >= 2)
    .dropDuplicates(partitionKeys)
    .withColumn("literature", array(col('pmid')))

    # Adding linteral columns:
    .withColumn('datasourceId',lit('europepmc'))
    .withColumn('datatypeId',lit('literature'))

    # Reorder columns:
    .select(["datasourceId", "datatypeId", "targetFromSource", "targetFromSourceId",'resourceScore',
            "diseaseFromSource","diseaseFromSourceMappedId","literature","textMiningSentences"])

    # Save output:
    .write.format('json').mode('overwrite').option('compression', 'gzip').save(outputFile)
)

Py4JJavaError: An error occurred while calling o151.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:226)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:178)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:175)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:213)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:210)
	at org.apache.

In [66]:
# old_file = 'old_epmc_sample.json'
old_file = '/Users/dsuveges/project_data/ot/evidence_input/21.02/epmc/cttv025-25-01-2021.json.gz'
# cooc_df = (
#     spark.read.json(old_file)
#     .filter(size(col('evidence.literature_ref.mined_sentences')) == 1)
#     .withColumn("sentences", explode(col('evidence.literature_ref.mined_sentences')))
#     .select(col('unique_association_fields.target_id').alias('target_id'), 
#         col('unique_association_fields.publication_id').alias('publication_id'), 
#         col('unique_association_fields.disease_id').alias('disease_id'), 
#         col('evidence.resource_score.value').alias('score'),
#         col('evidence.literature_ref.mined_sentences').alias('sentences'),
#         col('sentences.section').alias('section'))
#     .toPandas()
# )

# cooc_df.head()


import pandas as pd
import json
import gzip

old_data = []

with gzip.open(old_file) as f:
    for line in f:
        data = json.loads(line)
#         if len(data['evidence']['literature_ref']['mined_sentences']) > 1:
#             continue 
            
        old_data.append({
            'target': data['target']['id'].split('/')[-1],
            'disease': data['disease']['id'].split('/')[-1],
            'paper': data['evidence']['unique_experiment_reference'].split('/')[-1],
            'score': data['evidence']['resource_score']['value'],
            'first_section': data['evidence']['literature_ref']['mined_sentences'][0]['section'],
            'sections': [x['section'] for x in data['evidence']['literature_ref']['mined_sentences']]
        })

old_df = pd.DataFrame(old_data)
print(f'Number of evidence: {len(old_df)}')
print(old_df.head())

27
   target         disease     paper  score first_section  \
0  O14813     EFO_1001985  11882252   15.2      abstract   
1  O14813  Orphanet_45358  11882252   12.2         title   
2  P01108     EFO_0000621  11922865    8.0         other   
3  P01138     EFO_0000239  11922865    8.4      abstract   
4  P01138     EFO_0000621  11922865    6.4         other   

                                            sections  
0  [abstract, abstract, abstract, other, other, o...  
1                                     [title, other]  
2  [other, other, other, other, other, other, oth...  
3                                  [abstract, other]  
4                                     [other, other]  


In [67]:
len(old_df)

7976568

In [69]:
old_df.first_section.value_counts()

abstract    5629820
title       1417215
other        817758
figure        64473
table         46586
appendix        716
Name: first_section, dtype: int64

In [74]:
old_df.loc[old_df.score > 100]

,target,disease,paper,score,first_section,sections
7928,Q9H293,EFO_0005741,16606667,107.8,abstract,"[abstract, abstract, figure, figure, figure, f..."
10125,Q8N474,EFO_0000182,17626620,149.0,title,"[title, abstract, abstract, abstract, abstract..."
15221,P29590,Orphanet_647,17030982,193.8,abstract,"[abstract, abstract, figure, figure, figure, f..."
20021,P00519,EFO_0004289,15939795,135.6,abstract,"[abstract, figure, figure, figure, figure, fig..."
20030,P11274,EFO_0004289,15939795,135.6,abstract,"[abstract, figure, figure, figure, figure, fig..."
...,...,...,...,...,...,...
1431574,P23470,EFO_0003060,33174523,115.0,title,"[title, abstract, abstract, abstract, abstract..."
1432104,Q11201,EFO_0000756,33203881,106.0,title,"[title, abstract, abstract, abstract, abstract..."
1432532,Q9H7M9,MONDO_0007254,33250890,102.6,title,"[title, abstract, abstract, abstract, abstract..."
1436681,Q8NE86,EFO_0003897,33235465,101.6,abstract,"[abstract, abstract, abstract, abstract, abstr..."


In [79]:
filtered = old_df.loc[old_df.paper == '33203881',['target', 'disease', 'score', 'paper']]

In [1]:
import pyspark.sql
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import re

global spark

# SparkContext.setSystemProperty('spark.executor.memory', '20g')

spark = (pyspark.sql.SparkSession
    .builder
    .appName("phenodigm_parser")
    .config("spark.executor.memory", '10g')
     .config("spark.driver.bindAddress", "localhost")
    .config("spark.driver.memory", '10g')
    .getOrCreate()
)

#  

In [49]:
df = (
    spark.read.json('/Users/dsuveges/project_data/ot/evidence_input/21.04/phenodigm-2021-03-24.json.gz')
    .withColumn("biologicalModelId", 
                when(col("biologicalModelId").startswith("MGI"), col("biologicalModelId")).otherwise(None))
    .write.format('json').mode('overwrite').option('compression', 'gzip').save('/Users/dsuveges/project_data/ot/evidence_input/21.04/phenodigm-2021-03-24_b.json.gz')
)

In [12]:
(
    spark.read.json('/Users/dsuveges/Downloads/part-00000-2aa22b2a-5c07-48ae-919d-9cc14eaa7da6-c000.json.gz')
    .printSchema()
)

root
 |-- datasourceId: string (nullable = true)
 |-- datatypeId: string (nullable = true)
 |-- diseaseFromSource: string (nullable = true)
 |-- diseaseFromSourceMappedId: string (nullable = true)
 |-- literature: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- resourceScore: double (nullable = true)
 |-- targetFromSource: string (nullable = true)
 |-- targetFromSourceId: string (nullable = true)
 |-- textMiningSentences: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- dEnd: long (nullable = true)
 |    |    |-- dStart: long (nullable = true)
 |    |    |-- section: string (nullable = true)
 |    |    |-- tEnd: long (nullable = true)
 |    |    |-- tStart: long (nullable = true)
 |    |    |-- text: string (nullable = true)



In [7]:
df.show()

+--------------------+------------------+--------------------+
|                 col|targetFromSourceId|   diseaseFromSource|
+--------------------+------------------+--------------------+
|[, 108, 95, abstr...|   ENSG00000112715|      bladder tumors|
|[, 49, 35, abstra...|   ENSG00000112715|      bladder tumors|
|[, 83, 69, abstra...|   ENSG00000112715|      bladder tumors|
|[, 85, 69, title,...|   ENSG00000136997|    Burkitt lymphoma|
|[, 104, 88, abstr...|   ENSG00000136997|    Burkitt lymphoma|
|[, 183, 171, abst...|   ENSG00000118702|        hypoglycemia|
|[, 352, 349, abst...|   ENSG00000123384|acute myeloid leu...|
|[, 352, 349, abst...|   ENSG00000123384|acute myeloid leu...|
|[, 136, 114, titl...|   ENSG00000123384|acute myeloid leu...|
|[, 146, 143, abst...|   ENSG00000123384|acute myeloid leu...|
|[, 133, 130, abst...|   ENSG00000123384|acute myeloid leu...|
|[, 277, 274, abst...|   ENSG00000113161|                 CAD|
|[, 155, 144, abst...|   ENSG00000134184|         lung 

In [14]:
# (
#     spark.read.parquet('/Users/dsuveges/project_data/epmc_evidence/raw_epmc_cooc/part-00007-23c5c03e-7e4c-4e79-92da-e9bcb3eef403-c000.snappy.parquet')
#     .printSchema()
# )

root
 |-- pmid: string (nullable = true)
 |-- pubDate: string (nullable = true)
 |-- organisms: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- section: string (nullable = true)
 |-- text: string (nullable = true)
 |-- association: struct (nullable = true)
 |    |-- Altered_Expression: double (nullable = true)
 |    |-- Any: double (nullable = true)
 |    |-- Genetic_Variation: double (nullable = true)
 |    |-- Negative: double (nullable = true)
 |    |-- Neutral: double (nullable = true)
 |    |-- No: double (nullable = true)
 |    |-- Positive: double (nullable = true)
 |    |-- Regulatory_modification: double (nullable = true)
 |    |-- Yes: double (nullable = true)
 |-- end1: long (nullable = true)
 |-- end2: long (nullable = true)
 |-- evidence_score: double (nullable = true)
 |-- label1: string (nullable = true)
 |-- keywordId1: string (nullable = true)
 |-- label2: string (nullable = true)
 |-- keywordId2: string (nullable = true)
 |-- relation: arr

In [23]:



(
    spark.read.parquet('/Users/dsuveges/project_data/epmc_evidence/raw_epmc_cooc/part-00007-23c5c03e-7e4c-4e79-92da-e9bcb3eef403-c000.snappy.parquet')
    .filter(
        (col('type') == "GP-DS") & # Filtering for gene/disease cooccurrence
        (col('isMapped') == True) & # Filtering for mapping
        (col('pmid') != "") &  # Filtering out missing pmids
        (col('label1') != "(") & # Filtering out this strange entity.
        (length(col('text')) > 500 )) # Sentence threshold is 500 
    .withColumn('text_lenght', length(col('text')))
    .select(col('text_lenght'),col('text'))
    .dropDuplicates()
    .count()
)

444

In [45]:
(
    spark.read.parquet('/Users/dsuveges/project_data/epmc_evidence/raw_epmc_cooc/part-00007-23c5c03e-7e4c-4e79-92da-e9bcb3eef403-c000.snappy.parquet')
    .filter(
        (col('type') == "GP-DS") & # Filtering for gene/disease cooccurrence
        (col('isMapped') == True) & # Filtering for mapping
        (col('pmid') != "") &  # Filtering out missing pmids
        (col('label1') != "(") & # Filtering out this strange entity.
        (length(col('text')) > 500 )) # Sentence threshold is 500 
    .groupby([col('text'), col('pmid')])
    .agg(
        first(col("section")).alias('section'),
        length(col('text')).alias('textLenght'),
        collect_set(struct(
            col('keywordId1').alias('targetFromSourceId'),
            col("keywordId2").alias('diseaseFromSourceMappedId'),
        )).alias('associations'),
        collect_set(struct(
            col('label1').alias('targetLabel'),
            col("label2").alias('diseaseLabel'),
        )).alias('cooccurrences')
    )
    .withColumn('associationCount', size(col('associations')))
    .withColumn('cooccurrenceCount', size(col('cooccurrences')))

    .coalesce(1).write.format('json').mode('overwrite').option('compression', 'gzip')
    .save('epmc_long_sentences.json.gz')
)


In [47]:
df = spark.read.json('epmc-longSentences-600-2021-03-25.json.gz')




In [48]:
df.show()



+----------------+--------------------+-----------------+--------------------+----------+--------------------+--------------------+----------+
|associationCount|        associations|cooccurrenceCount|       cooccurrences|      pmid|             section|                text|textLenght|
+----------------+--------------------+-----------------+--------------------+----------+--------------------+--------------------+----------+
|               6|[[MONDO_0008199, ...|                2|[[PD, Aromatase],...|PMC6993383|characteristics o...|(%) (N = 971)Stud...|      3131|
|              12|[[EFO_0000673, EN...|                2|[[prostate cancer...|PMC5681753|             results|(%) (missing = 14...|      1184|
|              24|[[EFO_0010133, EN...|               20|[[hypoglycemia, C...|PMC4480895|baseline patient ...|(%) 1,5 AG: 1,5 a...|       892|
|               1|[[EFO_0000249, EN...|                1|          [[AD, Aβ]]|PMC6805864|participant chara...|(%)73 (27.14)25 (...|       668|

In [58]:
df_slim = (
    df
    .select(
        col('textLenght'),
        col('pmid'),
        col('associationCount'),
        col('cooccurrenceCount')
    )
)

df_slim.describe('textLenght')
                 
                 
                 

DataFrame[summary: string, textLenght: string]

In [59]:
df_slim.describe('textLenght').show()

+-------+------------------+
|summary|        textLenght|
+-------+------------------+
|  count|            271215|
|   mean|1114.1172575263167|
| stddev|1016.5778698172317|
|    min|               601|
|    max|             90390|
+-------+------------------+

